<a href="https://colab.research.google.com/github/vanderbilt-data-science/abstract-search/blob/main/abstract_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
import os
cur_path = "/content/gdrive/MyDrive/Transformers/"
os.chdir(cur_path)
!pwd

/content/gdrive/MyDrive/Transformers


In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 6.1 MB/s 
     |████████████████████████████████| 3.5 MB 50.4 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
     |████████████████████████████████| 6.8 MB 47.8 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=ce8f122f970bf9693d93bd221bb3cdfb5bf97fd5b5ca1e5541744d3ba11c8a22
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [25]:
# Load document embeddings
doc_emb = np.loadtxt("abstract-embed.txt", dtype=float)
doc_emb

array([[ 0.02671184, -0.00307606,  0.01811073, ...,  0.08794652,
        -0.01698122, -0.04554106],
       [-0.00468118, -0.06231824, -0.0260776 , ...,  0.11528901,
         0.02018194, -0.01833341],
       [-0.02323229,  0.03903552,  0.005878  , ...,  0.04989962,
        -0.0094997 , -0.00127161],
       ...,
       [-0.01831272, -0.02906192, -0.03881638, ..., -0.01849003,
         0.00707878, -0.03144346],
       [-0.04210049,  0.00465329, -0.01031839, ...,  0.04425324,
         0.01666035, -0.10110734],
       [ 0.00874317,  0.02460732, -0.10283821, ..., -0.05908803,
        -0.05419892,  0.01261293]])

In [17]:
# Load data
df = pd.read_csv("sessions.csv", usecols=['Unique ID', 'Name', 'Description', 'Activity Code', 'Start Time', 'End Time', 'Location Name'])
df.head()

,Unique ID,Name,Description,Activity Code,Start Time,End Time,Location Name
0,03D0227A-DCA8-474D-9677-414D3F780605,Keynote - 2021: The Tipping Point for Data Lit...,"A look at the rise of data literacy, and the c...",keynote1,2021-10-18 8:00,2021-10-18 8:45,Ballroom
1,AEDD11FF-C97B-4365-A66B-A92A38E64AC9,Leveraging Cognitive Diversity In Your Data Team,"What is Cognitive Diversity, and what does it ...",session015,2021-10-18 14:30,2021-10-18 15:30,Broadway B
2,35FB47CB-D26E-4AB0-A344-C52DF023C6FC,How to use data analytics to grow post-pandemi...,This presentation will focus on how to utilize...,workshop006,2021-10-18 9:00,2021-10-18 10:30,Broadway C
3,5085A26E-1739-432F-9472-BE4C1AF696D0,People Analytics: Using Data to Manage People ...,Most business planning processes focus on stra...,workshop005,2021-10-18 9:00,2021-10-18 10:30,Broadway B
4,6C12B9E4-1249-4675-A281-53C90D66BB5E,Get Your Model to Production Faster by Wrappin...,You've prepared the data and trained a model i...,workshop002,2021-10-18 9:00,2021-10-18 10:30,Cumberland 3 & 4


In [31]:
# Get attributes from dataframe
docs = list(df["Description"])
titles = list(df["Name"])
start_times = list(df["Start Time"])
end_times = list(df["End Time"])
locations = list(df["Location Name"])

In [19]:
# Load model 
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:

# Query
query = input("Enter your query: ")

#Encode query and documents
query_emb = model.encode(query).astype(float)

#Compute dot score between query and all document embeddings
scores = util.dot_score(query_emb, doc_emb.astype(float))[0].cpu().tolist()

#Combine docs & scores with other attributes
doc_score_pairs = list(zip(docs, scores, titles, start_times, end_times, locations))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)


#Output presentation recommendations
for doc, score, title, start_time, end_time, location in doc_score_pairs[:10]:
    print("Score: %f" %score)
    print("Title: %s" %title)
    print("Abstract: %s" %doc)
    print("Location: %s" %location)
    f"From {start_time} to {end_time}"
    print('\n')

Enter your query: people analytics
Score: 0.490518
Title: Managing an Analytics Platform on an Enterprise Scale for Multiple Business Units
Abstract: Are you looking to stand up an Analytics group for your Organization. What does the Operating model look like? How do you manage it at scale? What are the typical challenges you need to think through while evaluating potential tools? How do you manage boundaries of responsibility? I would be covering how we approach these questions within our Organization.
Location: Cumberland 5 & 6


Score: 0.475677
Title: Five Keys to Building a Successful Data and Analytics Team
Abstract: Healthcare analytics are advancing at a rapid pace making it difficult to keep up with changing technologies and methods. Leaders are thirsty for information that helps build their strategies and make data driven decisions. With all the talent in the market and advanced tools available, why is it that many data &amp; analytics efforts struggle to get off the ground an